# OpenAI API test

## Load keys

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY', 'YourAPIKey')


## Show installed packages

Check if openai packages etc are installed

In [3]:
# Uncomment to install required packages
#%pip install openai

In [5]:
%pip list

Package              Version
-------------------- --------
annotated-types      0.6.0
anyio                4.3.0
argon2-cffi          23.1.0
argon2-cffi-bindings 21.2.0
asttokens            2.4.1
bcrypt               4.1.2
certifi              2024.2.2
cffi                 1.16.0
click                8.1.7
colorama             0.4.6
comm                 0.2.2
cryptography         42.0.5
debugpy              1.6.7
decorator            5.1.1
distro               1.9.0
dnspython            2.6.1
ecdsa                0.18.0
email_validator      2.1.1
exceptiongroup       1.2.0
executing            2.0.1
fastapi              0.110.0
fastapi-jwt-auth     0.2.0
fastapi-pagination   0.12.21
fastapi-users        13.0.0
greenlet             3.0.3
h11                  0.14.0
httpcore             1.0.4
httpx                0.27.0
idna                 3.6
importlib_metadata   7.1.0
ipykernel            6.29.3
ipython              8.22.2
jedi                 0.19.1
jose                 1.0.0
jupyter

## First basic OpenAI code

In [6]:
from openai import OpenAI

Gets the key from the environment
Alternatively, can specify key using:

In [2]:

# client = OpenAI(
#     # This is the default and can be omitted
#     api_key="",
# )


#### Simple test call

In [7]:

client = OpenAI()

aiModel = "gpt-4-turbo-preview"

completion = client.chat.completions.create(
  model=aiModel,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the weather like?"}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content="I can't provide real-time weather updates or predictions. For the most current weather information, please check a trustworthy weather service website, a weather app, or a news outlet.", role='assistant', function_call=None, tool_calls=None)


For Async API Calls use await etc, rest is same

### Calling API for feedback with parameters formatted into message

#### Feedback prompt
<details>
<summary> System Prompt </summary>
Jij bent een leraar op een middelbare school voor een klas met studenten van ``[age]`` jaar oud met een ``[background educational info]``. Jij geeft het vak ``[course]``. Jij bent een expert in het domein van ``[course]``. 

Jij hebt een opdracht gegeven aan je studenten en je zal nu hun inzendingen verbeteren.

Hieronder zijn de originele instructies van de opdracht. De opdracht is afgebakend door `<start assignment>` en `<end assignment>`. Lees deze goed en begrijp ze:  
`<start assignment>`  
[assignment]  
`<end assignment>`  


Voor deze opdracht heb je toegang tot sjabloonoplossingen. De sjabloonoplossingen zijn telkens afgebakend door ``<start templatesolution>`` en ``<end templatesolution>``. De sjabloonoplossingen staan hieronder, lees deze goed en begrijp ze:  
``<start templatesolution>``  
[templatesolution]  
``<end templatesolution>``  

``<start templatesolution>``  
[templatesolution]  
``<end templatesolution>``  

``<start templatesolution>``  
[templatesolution]  
``<end templatesolution>``  

Gebruik deze sjabloonoplossingen om de inzending van de leerling te beoordelen door hun inzending te vergelijken met alle sjabloonoplossingen en feedback te geven die relevant is voor de oorspronkelijke opdracht. 

De student heeft geen toegang tot de sjabloonoplossingen en mag niet van het bestaan ervan op de hoogte zijn. Houd ze geheim en vermeld ze niet. 

Hieronder is de inzending van de student zijn werk. Lees deze goed en begrijp ze. Geef daarna feedback aan de student. De inzending is afgebakend door ``<start submission>`` en ``<end submissionn>``.

Houd de feedback die jij geeft kort. De student gebruikt de feedback om zijn eigen inzending te verbeteren en dient deze hier later opnieuw in. Maak niet zelf de opdracht voor de leerlingen. Houd de feedback eenvoudig en geef alleen hints. Voeg geen eigen voorbeelden toe aan je feedback of zinnen als "zoals...", de leerling moet zelf voorbeelden bedenken. 

Dit zal een leermoment zijn voor de student. Het doel is dat de student beter wordt in het vak [course].
</details>

<details>
<summary> User Prompt </summary>
Dit is de 1ste inzending van de student:

``<start submission>``  
[submission]  
``<end submissionn>``

</details>


Parameters needed are:
- age
- background educational info
- course
- assignment
- templatesolutions (list)
- submission

Use string Templates to substitute values in the messages

### Workflow: Hoe maak ik een system message?
* heb de parameters, geef die aan create system message
* die gaat intro template maken, en subtituten
* dan maak ik template solutions gedeelte, en substituten
* dan maak ik 3de outro template gedeelte, en substiten
* dan vul concat ik alles

In [8]:
import string
import textwrap

In [9]:

age = 12
background_educational_info = "redelijke wetenschappelijke opleiding"
course = "nederlands"

In [10]:
assignment = """**Opdracht 3: Vergelijkend Essay**

**Doel:** Het doel van deze opdracht is om je analytische vaardigheden te ontwikkelen door het vergelijken van twee literaire werken en het identificeren van overeenkomsten en verschillen in thema's, personages en plotontwikkeling.

**Taken:**
1. Kies één werk van "Van den vos Reynaerde" en kies één ander middeleeuws werk of een modern werk met vergelijkbare thema's als "Van den vos Reynaerde".
2. Lees beide werken grondig en maak aantekeningen over thema's, personages, plotontwikkeling, literaire stijl en andere relevante aspecten.
3. Schrijf een essay waarin je de twee werken vergelijkt. Zorg ervoor dat je:
- Een inleiding schrijft waarin je de werken kort introduceert en je vergelijkingsdoel uitlegt.
- Thema's bespreekt die in beide werken voorkomen en hoe ze worden behandeld.
- Personages vergelijkt en contrasten benadrukt in hun ontwikkeling, motivaties en interacties.
- Plotontwikkeling analyseert, inclusief belangrijke gebeurtenissen, climax en afloop.
- De literaire stijl van beide werken bespreekt, inclusief taalgebruik, verteltechnieken en symboliek.
- Tot een conclusie komt waarin je de belangrijkste bevindingen samenvat en je persoonlijke mening geeft over de waarde en impact van beide werken.

**Richtlijnen:**
- Vermeld duidelijk de titels en auteurs van beide werken die je hebt gekozen. Voor één van de gevallen "Van den vos Reynaerde" door Willem die Madocke maakte.
- Je essay moet minimaal 100 woorden bevatten.
- Gebruik specifieke voorbeelden en citaten uit beide werken om je analyse te ondersteunen.
- Houd rekening met de context en tijdperk waarin elk werk is geschreven.
- Wees kritisch en diepgaand in je analyse, vermijd oppervlakkige vergelijkingen.
- Zorg voor een duidelijke structuur met een inleiding, meerdere body paragraphs en een conclusie.
- Gebruik academische bronnen indien nodig om je argumenten te ondersteunen.

**Inleveren:**
- Zorg ervoor dat je je essay in een duidelijk en leesbaar formaat indient, bij voorkeur digitaal.

**Beoordeling:**
Je essay zal worden beoordeeld op de diepte van je analyse, de helderheid van je argumenten, de nauwkeurigheid van je interpretaties, en de algemene structuur en stijl van je schrijven. Creativiteit, originaliteit en het vermogen om kritisch te denken zullen ook worden gewaardeerd.

Deze opdracht biedt leerlingen de kans om hun analytische en schrijfvaardigheden te ontwikkelen door middel van een diepgaande vergelijkende analyse van literaire werken. Het is van belang dat de richtlijnen duidelijk zijn en dat de opdracht voldoende uitdaging biedt om de leerlingen te stimuleren tot een grondige analyse en reflectie."""

In [11]:
template_solutions = [
"""**Sjabloonoplossing 1: Vergelijkend Essay tussen "Van den vos Reynaerde" en "Animal Farm" door George Orwell**

**Inleiding**
In dit essay vergelijken we twee opmerkelijke literaire werken: "Van den vos Reynaerde" door Willem die Madocke maakte, en "Animal Farm" door George Orwell. Beide werken duiken in de complexe wereld van machtsstrijd en corruptie, maar elk doet dit op zijn unieke manier, gebruikmakend van dieren om menselijke ondeugden en politieke kritiek uit te beelden.

**Vergelijking van Thema's**
Beide "Van den vos Reynaerde" en "Animal Farm" draaien om thema's als corruptie, machtsmisbruik, en de strijd tegen onderdrukking. In "Van den vos Reynaerde" gebruikt Reynaert sluwheid en bedrog om uit handen van het gerecht te blijven, waardoor thema's als listigheid en rechtvaardigheid aan bod komen. "Animal Farm" brengt deze thema's naar een ander niveau door te laten zien hoe macht corrodeert en leidt tot tirannie, een weerspiegeling van de Stalinistische Sovjet-Unie.

**Personages Vergelijken**
Reynaert, de slinkse vos uit "Van den vos Reynaerde", en Napoleon, de autoritaire leider in "Animal Farm", geven elk hun eigen draai aan hoe macht kan worden misbruikt. Reynaert, hoewel vaak gezien als de schurk, speelt slim in op de zwakheden van anderen, terwijl Napoleon brute kracht en intimidatie gebruikt om zijn doelen te bereiken. Deze personages belichamen het concept van macht en manipulatie, hoewel Reynaert meer sluwheid vertoont en Napoleon een meer fysieke en autoritaire macht uitoefent.

**Plotontwikkeling Analyse**
De plotontwikkeling in beide boeken draait om de evolutie van macht en de consequenties van corruptie. "Van den vos Reynaerde" ontvouwt zich door de verschillende pogingen van Reynaert om gerechtigheid (door bedrog) te ontlopen, wat culmineert in zijn tijdelijke triomf. "Animal Farm", aan de andere kant, volgt de transformatie van een utopie tot een dystopie, waarbij de dieren ontdekken dat hun nieuwe heersers net zo corrupt zijn als degenen die ze hebben omvergeworpen. Beide verhalen eindigen met een reflectie op de cyclische aard van macht en corruptie.

**Literaire Stijl**
De literaire stijl van "Van den vos Reynaerde" kenmerkt zich door zijn satirische tonen, belichaamd in de antropomorfische dieren en hun interacties, wat voor humor zorgt zelfs als het donkere thema's verkent. "Animal Farm" deelt deze satirische en allegorische kwaliteiten, maar met een meer directe politieke boodschap. Beide werken maken gebruik van dieren als symbolen voor menselijk gedrag en sociale kritiek, maar doen dit op een manier die zowel vermakelijk als tot nadenken stemmend is.

**Conclusie**
"Van den vos Reynaerde" en "Animal Farm" zijn beide diepgaande kritieken op de menselijke natuur en politieke systemen, gebruikmakend van dieren om tijdloze thema's zoals corruptie, macht, en rechtvaardigheid te onderzoeken. Hoewel ze uit verschillende tijdperken komen en verschillende maatschappelijke kritieken aanvoeren, tonen beide werken de aantrekkingskracht en gevaren van macht, en hoe deze onvermijdelijk leidt tot corruptie en tirannie. De tijdloze waarde van beide werken benadrukt de noodzaak om waakzaam te blijven tegen machtsmisbruik en voor de eeuwige strijd voor rechtvaardigheid en gelijkheid.""", 

"""**Sjabloonoplossing 2: Vergelijkend Essay tussen "Van den vos Reynaerde" en "De Bende van Jan de Lichte" door Louis Paul Boon**

**Inleiding**
In dit essay richten we onze vergelijkende lens op "Van den vos Reynaerde," een middeleeuwse dierenfabel over avonturen en intriges, en "De Bende van Jan de Lichte," een historische roman door Louis Paul Boon die verhaalt over de lotgevallen van een 18e-eeuwse struikrover en zijn bende in Vlaanderen. Hoewel de werken uit verschillende tijdperken stammen, bevatten ze allebei scherpe maatschappelijke kritieken verpakt in hun verhalen.

**Vergelijking van Thema's**
Het centrale thema in beide boeken is onrecht en sociale ongelijkheid. "Van den vos Reynaerde" gebruikt dierlijke personages om de hypocrisie en corruptie binnen de middeleeuwse rechtssystemen aan de kaak te stellen, terwijl "De Bende van Jan de Lichte" de armoede, uitbuiting en het onvermogen van het gerechtelijk systeem in de 18e eeuw onderzoekt. Beide werken tonen een cynische kijk op de maatschappij en de machthebbers.

**Personages Vergelijken**
Hoewel Reynaert de vos en Jan de Lichte uit geheel andere werelden komen, delen ze een gemeenschappelijke sluwheid en een rebels karakter. Reynaert manipuleert iedereen om zich heen met zijn scherpe tong en intelligentie, terwijl Jan de Lichte het opneemt tegen de gevestigde orde met zijn moed en slimme plannen. Hun tegenspelers, vaak vertegenwoordigers van het gezag, worden in beide boeken geportretteerd als incompetent of corrupt, wat bijdraagt aan het gevoel van onrecht dat door de verhalen heen loopt.

**Plotontwikkeling Analyse**
"Van den vos Reynaerde" en "De Bende van Jan de Lichte" ontvouwen beide een narratief van vervolging en strijd tegen een corrupt systeem. Reynaert's strijd om te overleven tegen de krachten die hem willen bestraffen, echoot in de guerrillaoorlog van Jan de Lichte tegen de maatschappelijke structuren die hem en zijn bendeleden onderdrukken. Beide plotlijnen bouwen op naar een climax die de personages niet alleen fysiek, maar ook moreel uitdaagt, waarbij vragen over rechtvaardigheid en moraliteit naar voren komen.

**Literaire Stijl**
De stijl van Willem die Madocke maakte in "Van den vos Reynaerde" is beeldend en vol humor, waarmee hij de ernst van de kritieken verhult onder een laag van vermakelijke dierenfabels. Louis Paul Boon biedt in "De Bende van Jan de Lichte" een ruwer, directer realisme, met een focus op de donkere kanten van het menselijk bestaan. Beide auteurs gebruiken hun literaire technieken om kritiek te leveren op sociale ongelijkheid en machtsmisbruik, maar doen dit op merkbaar verschillende wijzen.

**Conclusie**
Hoewel "Van den vos Reynaerde" en "De Bende van Jan de Lichte" uit verschillende perioden stammen en verschillende genres vertegenwoordigen, delen ze een kritische blik op de maatschappij en de machtsverhoudingen binnen. Beide werken stellen vragen over rechtvaardigheid, macht en corruptie, en leveren zo kritiek op de maatschappelijke orde van hun tijd. Deze tijdloze thema's benadrukken de continue relevantie van beide werken in de discussie over sociale rechtvaardigheid en het menselijk gedrag.""", 

"""**Sjabloonoplossing 3: Vergelijkend Essay tussen "Van den vos Reynaerde" en "Het gouden ei" van Tim Krabbé**

**Inleiding**
Dit essay biedt een vergelijkend onderzoek tussen "Van den vos Reynaerde," een klassiek werk uit de Middelnederlandse literatuur dat de schalkse avonturen van een vos belicht, en "Het gouden ei" van Tim Krabbé, een moderne psychologische thriller. Ondanks de afwijkende genres en historische context, belichten beide werken thema's van bedrog, rechtvaardigheid en de donkere kant van het menselijk (of dierlijk) streven.

**Vergelijking van Thema's**
In beide werken is de sluwe manipulatie en het bedrog een centraal thema. "Van den vos Reynaerde" gebruikt deze thematiek om de corruptie binnen het justitiële systeem en de maatschappij bloot te leggen, terwijl "Het gouden ei" de obsessieve speurtocht naar de waarheid en de duistere gevolgen van obsessie en bedrog verkent. Beide verhalen stellen de vraag naar wat rechtvaardigheid echt betekent.

**Personages Vergelijken**
Reynaert en Raymond Lemorne, de antagonist in "Het gouden ei," vertegenwoordigen twee gezichten van kwaadaardigheid en sluwheid. Hoewel Reynaert met zijn listen en lagen enige sympathie kan wekken door zijn vindingrijkheid, is Raymond koud en berekenend, met een duisternis die diep geworteld is in zijn psyche. De complexiteit van deze karakters dwingt de lezer om te reflecteren op de vraag van moraliteit en of het doel de middelen heiligt.

**Plotontwikkeling Analyse**
De verhaallijnen van beide boeken zijn opvallend verschillend in constructie maar vergelijkbaar in de kern van manipulatie en het ontrafelen van een 'mysterie'. In "Van den vos Reynaerde" worden de verschillende lagen van bedrog en politieke intrige ontvouwd in een middeleeuwse setting, terwijl "Het gouden ei" de diepe emotionele impact en psychologische diepten van zijn personages door een moderne thrillerlaag verkent. Beide verhalen houden de lezer in spanning met onverwachte plotwendingen.

**Literaire Stijl**
De literaire stijl van beide werken draagt bij aan hun thematische diepgang. De middeleeuwse tekst van "Van den vos Reynaerde" biedt een satirische, soms bijna speelse benadering van serieuze maatschappelijke vraagstukken, terwijl "Het gouden ei" zich kenmerkt door strak, beknopt proza dat spanning en onbehagen oproept. Deze stijlkeuzes versterken de manier waarop elk verhaal zijn thema’s exploreert en presenteren een rijk spectrum aan technieken voor het overbrengen van spanning en morele dilemma’s.

**Conclusie**
"Van den vos Reynaerde" en "Het gouden ei" bieden beide een intrigerende blik op thema's als bedrog, rechtvaardigheid en de aard van kwaad, alhoewel door compleet verschillende lenzen. Wat deze werken verbindt, is hun vermogen om de lezer te laten reflecteren op de complexiteit van morele vraagstukken en de duistere kanten van menselijk gedrag, ongeacht de tijd of setting. Deze tijdloze thema’s resoneren ook vandaag de dag nog en verklaren de blijvende impact en relevantie van beide werken binnen hun literaire tradities."""
]

In [12]:
submission = "Blabla ik ga deze opdracht niet doen, wat een stom vak!!!!"

#### Indents niet veranderen voor multiline strings, anders output die tabs voor elke lijn... 


In [13]:
def create_system_message_intro(age: int, background_educational_info: string, course: string, assignment: string) -> string:
    intro = string.Template("""Jij bent een leraar op een middelbare school voor een klas met studenten van ${age} jaar oud met een ${background_educational_info}. Jij geeft het vak ${course}. Jij bent een expert in het domein van ${course}.

Jij hebt een opdracht gegeven aan je studenten en je zal nu hun inzendingen verbeteren.

Hieronder zijn de originele instructies van de opdracht. De opdracht is afgebakend door <start assignment> en <end assignment>. Lees deze goed en begrijp ze:

<start assignment>
${assignment}
<end assignment>


Voor deze opdracht heb je toegang tot sjabloonoplossingen. De sjabloonoplossingen zijn telkens afgebakend door <start templatesolution> en <end templatesolution>. De sjabloonoplossingen staan hieronder, lees deze goed en begrijp ze:
""")
    
    intro = intro.substitute(age=age, background_educational_info=background_educational_info, course=course, assignment=assignment)

    return textwrap.dedent(intro)

In [14]:
def create_system_message_templates(template_solutions: list) -> string:
  templates = ""

  for solution in template_solutions:
    template = string.Template("""<start templatesolution>
${solution}
<end templatesolution>

""")
      
    templates += template.substitute(solution=solution);
  
  return textwrap.dedent(templates)

In [17]:
def create_system_message_outro(course: string) -> string:
  outro = string.Template("""Gebruik deze sjabloonoplossingen om de inzending van de leerling te beoordelen door hun inzending te vergelijken met alle sjabloonoplossingen en feedback te geven die relevant is voor de oorspronkelijke opdracht. 

De student heeft geen toegang tot de sjabloonoplossingen en mag niet van het bestaan ervan op de hoogte zijn. Houd ze geheim en vermeld ze niet. 

Hieronder is de inzending van de student zijn werk. Lees deze goed en begrijp ze. Geef daarna feedback aan de student. De inzending is afgebakend door <start submission> en <end submissionn>.

Houd de feedback die jij geeft kort. De student gebruikt de feedback om zijn eigen inzending te verbeteren en dient deze hier later opnieuw in. Maak niet zelf de opdracht voor de leerlingen. Houd de feedback eenvoudig en geef alleen hints. Voeg geen eigen voorbeelden toe aan je feedback of zinnen als "zoals...", de leerling moet zelf voorbeelden bedenken.

Dit zal een leermoment zijn voor de student. Het doel is dat de student beter wordt in het vak ${course}.""")

  return textwrap.dedent(outro.substitute(course=course))

In [19]:
def create_system_message(
    age: int, 
    background_educational_info: string, 
    course: string, 
    assignment: string,
    template_solutions: list):
  intro = create_system_message_intro(age, background_educational_info, course, assignment)
  # print("---\n"+intro)
  templates = create_system_message_templates(template_solutions)
  # print("---\n"+templates)
  outro = create_system_message_outro(course=course)
  # print("---\n"+outro)
  
  return intro + templates + outro


system_message = create_system_message(age=age, 
  background_educational_info=background_educational_info, 
  course=course, assignment=assignment, 
  template_solutions=template_solutions)
print(system_message)


Jij bent een leraar op een middelbare school voor een klas met studenten van 12 jaar oud met een redelijke wetenschappelijke opleiding. Jij geeft het vak nederlands. Jij bent een expert in het domein van nederlands.

Jij hebt een opdracht gegeven aan je studenten en je zal nu hun inzendingen verbeteren.

Hieronder zijn de originele instructies van de opdracht. De opdracht is afgebakend door <start assignment> en <end assignment>. Lees deze goed en begrijp ze:

<start assignment>
**Opdracht 3: Vergelijkend Essay**

**Doel:** Het doel van deze opdracht is om je analytische vaardigheden te ontwikkelen door het vergelijken van twee literaire werken en het identificeren van overeenkomsten en verschillen in thema's, personages en plotontwikkeling.

**Taken:**
1. Kies één werk van "Van den vos Reynaerde" en kies één ander middeleeuws werk of een modern werk met vergelijkbare thema's als "Van den vos Reynaerde".
2. Lees beide werken grondig en maak aantekeningen over thema's, personages, ploto

### Workflow: Hoe maak ik de user message?
* heb de assignment nodig, en subtitute die in een tempalte

In [21]:
def create_user_message(
    submission: string):

  # Oftewel "1ste inzending", oftewel altijd "laatste inzending" schrijven altijd...
  user_message = string.Template("""Dit is de laatste inzending van de student:
<start submission>
${submission}
<end submission>""")

  return user_message.substitute(submission=submission)


user_message = create_user_message(submission=submission)
print(user_message)

Dit is de laatste inzending van de student:
<start submission>
Blabla ik ga deze opdracht niet doen, wat een stom vak!!!!
<end submission>


### Maak de API call met een de feedback prompt en een echte submission

In [22]:
# Test the API call with formatted messages
completion = client.chat.completions.create(
  model=aiModel,
  messages=[
    {"role": "system", "content": create_system_message(
        age=age, 
        background_educational_info=background_educational_info, 
        course=course, assignment=assignment, 
        template_solutions=template_solutions)},
    {"role": "user", "content": create_user_message(submission=submission)}
  ]
)

print(completion.choices[0].message)

# Next Step: Reading from message history

will be done in the actual project itself

# Next Step: Doing the above with Langchain